In [1]:
import numpy as np
import time
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.decomposition import KernelPCA

In [2]:
# Load or create dataset with more than 2 dimensions
X, y = make_classification(n_features=6, n_classes=2, random_state=0, n_samples=10000)

In [3]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [4]:
#Build a pipeline to tune the hyperparameters of Kernel PCA and also the neural network
from sklearn.pipeline import Pipeline
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(optimizer="adam"):
    model = Sequential()
    model.add(Dense(units=32, activation="relu"))
    model.add(Dense(units=1, activation="sigmoid"))
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Create the pipeline
pipeline = Pipeline([("kpca", KernelPCA()), ("classifier", KerasClassifier(build_fn=create_model))])

# Define the hyperparameters to tune
param_grid = {
    "kpca__kernel": ["linear", "sigmoid", "rbf"],
    "kpca__n_components": [2, 3],
    "classifier__optimizer": ["adam", "sgd"],
    "classifier__epochs": [50, 100],
    "classifier__batch_size": [32, 64]
}

# Tune the hyperparameters using grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=3)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

<ipython-input-4-ed73f69866f8>:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  pipeline = Pipeline([("kpca", KernelPCA()), ("classifier", KerasClassifier(build_fn=create_model))])


Epoch 1/50
167/167 [==============================] - 8s 3ms/step - loss: 0.5535 - accuracy: 0.7150
Epoch 2/50
167/167 [==============================] - 1s 3ms/step - loss: 0.4038 - accuracy: 0.8342
Epoch 3/50
167/167 [==============================] - 1s 3ms/step - loss: 0.3872 - accuracy: 0.8384
Epoch 4/50
167/167 [==============================] - 1s 3ms/step - loss: 0.3774 - accuracy: 0.8417
Epoch 5/50
167/167 [==============================] - 1s 3ms/step - loss: 0.3673 - accuracy: 0.8476
Epoch 6/50
167/167 [==============================] - 1s 3ms/step - loss: 0.3578 - accuracy: 0.8549
Epoch 7/50
167/167 [==============================] - 1s 3ms/step - loss: 0.3478 - accuracy: 0.8584
Epoch 8/50
  1/167 [..............................] - ETA: 0s - loss: 0.3128 - accuracy: 0.8750

KeyboardInterrupt: ignored

In [ ]:
 grid_search.best_score_

Same for MNIST

In [5]:
# Loading libraries
import tensorflow as tf
from tensorflow import keras

# Loading the data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Split 5000 instances to the validation set and scale the 8 bit images in range from 0 to 1
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

input_shape=X_train[0].shape
def build_model(n_hidden = 1, n_neurons = 30, activation_func="relu", input_shape=input_shape):
    grid_model = keras.models.Sequential()
    grid_model.add(keras.layers.Flatten(input_shape=input_shape))
    for layer in range(n_hidden):
        grid_model.add(keras.layers.Dense(n_neurons, activation=activation_func))
    grid_model.add(keras.layers.Dense(10, activation="softmax"))
    opt = tf.keras.optimizers.Adam()
    grid_model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return grid_model

In [7]:
param_distribs = {
    "n_hidden": [1, 2, 3, 4, 5],
    "n_neurons": np.arange(1, 300),
    "activation_func": ["relu", "sigmoid", "tanh"]
}

In [8]:
keras_clf = KerasClassifier(build_model, n_hidden=param_distribs["n_hidden"], n_neurons=param_distribs["n_neurons"],
                           activation_func=param_distribs["activation_func"], verbose=1)

<ipython-input-8-0975dc5aa10b>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_clf = KerasClassifier(build_model, n_hidden=param_distribs["n_hidden"], n_neurons=param_distribs["n_neurons"],


In [9]:
rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=10, cv=3)

rnd_search_cv.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), 
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/30
1146/1146 [==============================] - 6s 4ms/step - loss: 0.5486 - accuracy: 0.8099 - val_loss: 0.4613 - val_accuracy: 0.8326
Epoch 2/30
1146/1146 [==============================] - 4s 3ms/step - loss: 0.4190 - accuracy: 0.8522 - val_loss: 0.4030 - val_accuracy: 0.8590
Epoch 3/30
1146/1146 [==============================] - 5s 4ms/step - loss: 0.3755 - accuracy: 0.8670 - val_loss: 0.3830 - val_accuracy: 0.8608
Epoch 4/30
1146/1146 [==============================] - 4s 3ms/step - loss: 0.3460 - accuracy: 0.8762 - val_loss: 0.3460 - val_accuracy: 0.8818
Epoch 5/30
1146/1146 [==============================] - 6s 5ms/step - loss: 0.3272 - accuracy: 0.8818 - val_loss: 0.3595 - val_accuracy: 0.8766
Epoch 6/30
1146/1146 [==============================] - 9s 7ms/step - loss: 0.3120 - accuracy: 0.8873 - val_loss: 0.3307 - val_accuracy: 0.8852
Epoch 7/30
1146/1146 [==============================] - 6s 5ms/step - loss: 0.2953 - accuracy: 0.8915 - val_loss: 0.3459 - val_accuracy:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7e140d0400>,
                   param_distributions={'activation_func': ['relu', 'sigmoid',
                                                            'tanh'],
                                        'n_hidden': [1, 2, 3, 4, 5],
                                        'n_neurons': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,...
       209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
       222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234,
       235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247,
       248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260,
       261, 262, 263, 264, 265, 266

In [10]:
rnd_search_cv.best_score_

0.8896727363268534

In [12]:
# Print the best hyperparameters
print("Best Hyperparameters:", rnd_search_cv.best_params_)

Best Hyperparameters: {'n_neurons': 181, 'n_hidden': 1, 'activation_func': 'relu'}
